In [9]:
import numpy as np
import os
import shutil
import subprocess



In [10]:
total_count = 8


replica_count = 4

n_clusters = int(total_count / replica_count)

nclients = 2

In [11]:
for folder in os.listdir('.'):
    if 'config' in folder and folder!='config' and 'zip' not in folder and '.' not in folder:
        shutil.rmtree(folder)

In [12]:
os.system("aws --region us-west-1 ec2 describe-instances --filter Name=instance-type,Values=t2.small --query 'Reservations[*].Instances[*].[PrivateIpAddress]' --output text > all_internal_ips")

os.system("aws --region us-west-1 ec2 describe-instances  --filter Name=instance-type,Values=t2.small --query 'Reservations[*].Instances[*].[PublicIpAddress]' --output text > all_external_ips")


0

In [13]:
with open('all_internal_ips') as f:
    ipList = f.read().splitlines()
ipList = [x for x in ipList if x!='None']

In [14]:
ipList, len(ipList)

(['172.31.30.253',
  '172.31.19.141',
  '172.31.29.13',
  '172.31.24.173',
  '172.31.29.60',
  '172.31.20.218',
  '172.31.17.166',
  '172.31.26.133',
  '172.31.28.18',
  '172.31.30.33'],
 10)

In [15]:
clusterIpMap = {}

for i in range(len(ipList)):
    clusterIpMap[i] = ipList[i]

    
for i in range(nclients):
    
    clusterIpMap[7001+i] = ipList[-(nclients-i)]




clusterPortMap1 = {}
clusterPortMap2 = {}

for i in range(len(ipList) - nclients):
    
    clusterPortMap1[i] = str(11000+i*10)
    
for i in range(len(ipList) - nclients):
    clusterPortMap2[i] = str(11000+i*10+1)
    
# clusterPortMap1[7003] = '11400'
# clusterPortMap2[7003] = '11410'



temp = 11900

for i in range(nclients):
    
    clusterPortMap1[7001+i] = str(temp+i*20)
    clusterPortMap2[7001+i] = str(temp+i*20+10)



In [16]:
clusterPortMap1, clusterPortMap2

({0: '11000',
  1: '11010',
  2: '11020',
  3: '11030',
  4: '11040',
  5: '11050',
  6: '11060',
  7: '11070',
  7001: '11900',
  7002: '11920'},
 {0: '11001',
  1: '11011',
  2: '11021',
  3: '11031',
  4: '11041',
  5: '11051',
  6: '11061',
  7: '11071',
  7001: '11910',
  7002: '11930'})

In [17]:
os.system('./gradlew installDist')
for i in range(total_count):
    os.system('./runscripts/smartrun.sh bftsmart.tom.util.RSAKeyPairGenerator '+str(i)+' 1024')

    

os.system('./runscripts/smartrun.sh bftsmart.tom.util.RSAKeyPairGenerator '+str(7002)+' 1024')
os.system('./runscripts/smartrun.sh bftsmart.tom.util.RSAKeyPairGenerator '+str(7001)+' 1024')


os.system('./gradlew clean')


Starting a Gradle Daemon, 1 incompatible and 1 stopped Daemons could not be reused, use --status for details


Failed to execute org.gradle.cache.internal.AsyncCacheAccessDecoratedCache$$Lambda$308/0x000000084040c440@4ebea69e.
org.gradle.api.UncheckedIOException: Could not add entry '/home/tejas/work/BftSmart_Final/build/classes/java/main/bftsmart/demo/ycsb/Task.class' to cache fileHashes.bin (/home/tejas/work/BftSmart_Final/.gradle/7.1/fileHashes/fileHashes.bin).
	at org.gradle.cache.internal.btree.BTreePersistentIndexedCache.put(BTreePersistentIndexedCache.java:162)
	at org.gradle.cache.internal.DefaultMultiProcessSafePersistentIndexedCache.lambda$put$1(DefaultMultiProcessSafePersistentIndexedCache.java:67)
	at org.gradle.cache.internal.DefaultFileLockManager$DefaultFileLock.doWriteAction(DefaultFileLockManager.java:216)
	at org.gradle.cache.internal.DefaultFileLockManager$DefaultFileLock.writeFile(DefaultFileLockManager.java:206)
	at org.gradle.cache.internal.DefaultCacheAccess$UnitOfWorkFileAccess.writeFile(DefaultCacheAccess.java:449)
	at org.gradle.cache.internal.DefaultMultiProcessSafePe

org.gradle.api.UncheckedIOException: Could not add entry '/home/tejas/work/BftSmart_Final/build/classes/java/main/bftsmart/communication/server/Test.class' to cache fileHashes.bin (/home/tejas/work/BftSmart_Final/.gradle/7.1/fileHashes/fileHashes.bin).
	at org.gradle.cache.internal.btree.BTreePersistentIndexedCache.put(BTreePersistentIndexedCache.java:162)
	at org.gradle.cache.internal.DefaultMultiProcessSafePersistentIndexedCache.lambda$put$1(DefaultMultiProcessSafePersistentIndexedCache.java:67)
	at org.gradle.cache.internal.DefaultFileLockManager$DefaultFileLock.doWriteAction(DefaultFileLockManager.java:216)
	at org.gradle.cache.internal.DefaultFileLockManager$DefaultFileLock.writeFile(DefaultFileLockManager.java:206)
	at org.gradle.cache.internal.DefaultCacheAccess$UnitOfWorkFileAccess.writeFile(DefaultCacheAccess.java:449)
	at org.gradle.cache.internal.DefaultMultiProcessSafePersistentIndexedCache.put(DefaultMultiProcessSafePersistentIndexedCache.java:67)
	at org.gradle.cache.inte


> Task :compileJava



org.gradle.api.UncheckedIOException: Could not add entry '/home/tejas/work/BftSmart_Final/build/classes/java/main/bftsmart/tom/ServiceProxy$1.class' to cache fileHashes.bin (/home/tejas/work/BftSmart_Final/.gradle/7.1/fileHashes/fileHashes.bin).
	at org.gradle.cache.internal.btree.BTreePersistentIndexedCache.put(BTreePersistentIndexedCache.java:162)
	at org.gradle.cache.internal.DefaultMultiProcessSafePersistentIndexedCache.lambda$put$1(DefaultMultiProcessSafePersistentIndexedCache.java:67)
	at org.gradle.cache.internal.DefaultFileLockManager$DefaultFileLock.doWriteAction(DefaultFileLockManager.java:216)
	at org.gradle.cache.internal.DefaultFileLockManager$DefaultFileLock.writeFile(DefaultFileLockManager.java:206)
	at org.gradle.cache.internal.DefaultCacheAccess$UnitOfWorkFileAccess.writeFile(DefaultCacheAccess.java:449)
	at org.gradle.cache.internal.DefaultMultiProcessSafePersistentIndexedCache.put(DefaultMultiProcessSafePersistentIndexedCache.java:67)
	at org.gradle.cache.internal.As

...

org.gradle.api.UncheckedIOException: Could not add entry '/home/tejas/work/BftSmart_Final/build/install/library/config/keysECDSA/publickey15' to cache fileHashes.bin (/home/tejas/work/BftSmart_Final/.gradle/7.1/fileHashes/fileHashes.bin).
	at org.gradle.cache.internal.btree.BTreePersistentIndexedCache.put(BTreePersistentIndexedCache.java:162)
	at org.gradle.cache.internal.DefaultMultiProcessSafePersistentIndexedCache.lambda$put$1(DefaultMultiProcessSafePersistentIndexedCache.java:67)
	at org.gradle.cache.internal.DefaultFileLockManager$DefaultFileLock.doWriteAction(DefaultFileLockManager.java:216)
	at org.gradle.cache.internal.DefaultFileLockManager$DefaultFileLock.writeFile(DefaultFileLockManager.java:206)
	at org.gradle.cache.internal.DefaultCacheAccess$UnitOfWorkFileAccess.writeFile(DefaultCacheAccess.java:449)
	at org.gradle.cache.internal.DefaultMultiProcessSafePersistentIndexedCache.put(DefaultMultiProcessSafePersistentIndexedCache.java:67)
	at org.gradle.cache.internal.AsyncCach

Failed to execute org.gradle.cache.internal.AsyncCacheAccessDecoratedCache$$Lambda$308/0x000000084040c440@286c662.
org.gradle.api.UncheckedIOException: Could not add entry '/home/tejas/work/BftSmart_Final/build/install/library/smartrun.cmd' to cache fileHashes.bin (/home/tejas/work/BftSmart_Final/.gradle/7.1/fileHashes/fileHashes.bin).
	at org.gradle.cache.internal.btree.BTreePersistentIndexedCache.put(BTreePersistentIndexedCache.java:162)
	at org.gradle.cache.internal.DefaultMultiProcessSafePersistentIndexedCache.lambda$put$1(DefaultMultiProcessSafePersistentIndexedCache.java:67)
	at org.gradle.cache.internal.DefaultFileLockManager$DefaultFileLock.doWriteAction(DefaultFileLockManager.java:216)
	at org.gradle.cache.internal.DefaultFileLockManager$DefaultFileLock.writeFile(DefaultFileLockManager.java:206)
	at org.gradle.cache.internal.DefaultCacheAccess$UnitOfWorkFileAccess.writeFile(DefaultCacheAccess.java:449)
	at org.gradle.cache.internal.DefaultMultiProcessSafePersistentIndexedCache

> Task :installDist

BUILD SUCCESSFUL in 7s
3 actionable tasks: 3 executed



FAILURE: Build failed with an exception.

* What went wrong:
Gradle could not start your build.
> Could not create service of type BuildModelController using BuildScopeServicesProvider.createBuildModelController().
   > Could not create service of type FileHasher using BuildSessionServices.createFileHasher().
      > Cannot lock file hash cache (/home/tejas/work/BftSmart_Final/.gradle/7.1/fileHashes) as it has already been locked by this process.

* Try:
Run with --stacktrace option to get the stack trace. Run with --info or --debug option to get more log output. Run with --scan to get full insights.

* Get more help at https://help.gradle.org

BUILD FAILED in 463ms


256

In [18]:
for key in clusterPortMap1.keys():
    if key>7002:
        print(key)
        print()
        
        for folder in os.listdir('config/'):
            if os.path.isdir('config/'+folder) and 'keys' in folder:
                if os.path.exists('config/'+folder+'/'+'privatekey7001'):
                    shutil.copy('config/'+folder+'/'+'privatekey7001','config/'+folder+'/'+'privatekey'+str(key))
                    shutil.copy('config/'+folder+'/'+'publickey7001','config/'+folder+'/'+'publickey'+str(key))
                
        print('----------')
        print()

In [19]:
for i in range(n_clusters):
    
    if os.path.exists('config'+str(i)):
        shutil.rmtree('config'+str(i))
    
    os.mkdir('config'+str(i))
    os.mkdir('config'+str(i)+'/keysRSA')
    os.mkdir('config'+str(i)+'/keysSunEC')
    os.mkdir('config'+str(i)+'/keysECDSA')
    
    
    shutil.copytree('config/workloads/', 'config'+str(i)+'/workloads')
    shutil.copytree('config/keysSSL_TLS/', 'config'+str(i)+'/keysSSL_TLS')
    
    
    for file in os.listdir('config/keys/'):
    
        shutil.copy('config/keys/'+file, 'config'+str(i)+'/keysRSA/'+file)
        shutil.copy('config/keys/'+file, 'config'+str(i)+'/keysSunEC/'+file)
        shutil.copy('config/keys/'+file, 'config'+str(i)+'/keysECDSA/'+file)
    
    
    shutil.copy('config/system.config', 'config'+str(i)+'/system.config')    
    shutil.copy('config/logback.xml', 'config'+str(i)+'/logback.xml')
    
    
    
    

In [20]:
        
        for i in range(nclients):
            print(str(7001+i)+' '+clusterIpMap[7001+i]+' '+clusterPortMap1[7001+i]+'\n')

7001 172.31.28.18 11900

7002 172.31.30.33 11920



In [21]:
for n_cluster in range(n_clusters):
    
    with open('config'+str(n_cluster)+'/hosts.config','w') as file:
        
        for n_cluster1 in range(n_clusters):
        
            for i_replica in range(replica_count):
    #             file.writelines(str(i_replica)+' '+clusterIpMap[n_cluster*replica_count+i_replica]+' '  +clusterPortMap1[n_cluster*replica_count+i_replica]+' ' +  clusterPortMap2[n_cluster*replica_count+i_replica]+'\n')
                file.writelines(str(n_cluster1*replica_count+i_replica)+' '+clusterIpMap[n_cluster1*replica_count+i_replica]+' '  +clusterPortMap1[n_cluster1*replica_count+i_replica]+' ' +  clusterPortMap2[n_cluster1*replica_count+i_replica]+' '+str(n_cluster1)+'\n')

        for iter_ in range(len(ipList) - (n_clusters*replica_count)-nclients):
            print((n_clusters*replica_count) + iter_)
            
#             file.writelines(str((n_clusters*replica_count) + iter_)+' '+clusterIpMap[(n_clusters*replica_count) + iter_]+' '  +clusterPortMap1[(n_clusters*replica_count) + iter_]+' ' +  clusterPortMap2[(n_clusters*replica_count) + iter_]+' '+str(iter_%n_clusters)+'\n')
            file.writelines(str((n_clusters*replica_count) + iter_)+' '+clusterIpMap[(n_clusters*replica_count) + iter_]+' '  +clusterPortMap1[(n_clusters*replica_count) + iter_]+' ' +  clusterPortMap2[(n_clusters*replica_count) + iter_]+' '+str(0%n_clusters)+'\n')
        
        for i in range(nclients):
            file.writelines(str(7001+i)+' '+clusterIpMap[7001+i]+' '+clusterPortMap1[7001+i]+'\n')

        
        file.close()


In [22]:
replica_count

4

In [23]:
for n_cluster in range(n_clusters):
    with open('config'+str(n_cluster)+'/system.config','r') as file:
        data = file.readlines()
        f = int(np.ceil((replica_count-1)/3))
        print('f is',f )
        
        for n_line in range(len(data)):
            if ('system.servers.num' in data[n_line]) and '#' not in data[n_line]:
                
                data[n_line] = 'system.servers.num = '+str(replica_count)
                
            if ('system.servers.f' in data[n_line]) and '#' not in data[n_line]:
                data[n_line] = 'system.servers.f = '+str(f)
                
            if ('system.initial.view' in data[n_line]) and '#' not in data[n_line]:
                data[n_line] = 'system.initial.view = '
                for iter_view in range(replica_count):
                    data[n_line] = data[n_line] +str(n_cluster*replica_count+iter_view)+','
#                     data[n_line] = data[n_line] +str(n_cluster*replica_count+iter_view)+','
                
                data[n_line] = data[n_line][:-1]
                
                
        file.close()
        
    with open('config'+str(n_cluster)+'/system.config', 'w') as f:
        for line in data:
            f.write(f"{line}\n")
        

f is 1
f is 1


In [24]:
for n_cluster in range(n_clusters):
    for i in range(11):
        
        for ncls in  range(1):
        
        
            print('config'+str(n_cluster)+'/keysRSA/'+'publickey7002', \
                        'config'+str(n_cluster)+'/keysRSA/'+'publickey'+str(1000+(ncls*100)+i))
            shutil.copy('config'+str(n_cluster)+'/keysRSA/'+'publickey7002', \
                        'config'+str(n_cluster)+'/keysRSA/'+'publickey'+str(1000+(ncls*100)+i))
            shutil.copy('config'+str(n_cluster)+'/keysRSA/'+'privatekey7002', \
                        'config'+str(n_cluster)+'/keysRSA/'+'privatekey'+str(1000+(ncls*100)+i))
        
#         shutil.copy('config'+str(n_cluster)+'/keysRSA/'+'publickey7002', \
#                     'config'+str(n_cluster)+'/keysRSA/'+'publickey7002')
        
#         shutil.copy('config'+str(n_cluster)+'/keysRSA/'+'privatekey7002', \
#                     'config'+str(n_cluster)+'/keysRSA/'+'privatekey7002')

config0/keysRSA/publickey7002 config0/keysRSA/publickey1000
config0/keysRSA/publickey7002 config0/keysRSA/publickey1001
config0/keysRSA/publickey7002 config0/keysRSA/publickey1002
config0/keysRSA/publickey7002 config0/keysRSA/publickey1003
config0/keysRSA/publickey7002 config0/keysRSA/publickey1004
config0/keysRSA/publickey7002 config0/keysRSA/publickey1005
config0/keysRSA/publickey7002 config0/keysRSA/publickey1006
config0/keysRSA/publickey7002 config0/keysRSA/publickey1007
config0/keysRSA/publickey7002 config0/keysRSA/publickey1008
config0/keysRSA/publickey7002 config0/keysRSA/publickey1009
config0/keysRSA/publickey7002 config0/keysRSA/publickey1010
config1/keysRSA/publickey7002 config1/keysRSA/publickey1000
config1/keysRSA/publickey7002 config1/keysRSA/publickey1001
config1/keysRSA/publickey7002 config1/keysRSA/publickey1002
config1/keysRSA/publickey7002 config1/keysRSA/publickey1003
config1/keysRSA/publickey7002 config1/keysRSA/publickey1004
config1/keysRSA/publickey7002 config1/ke

In [26]:
os.system('git remote set-url origin https://github.com/tmane002/BftSmart_Final.git')

0

In [58]:
os.system('git add .; git commit -m "Working with modified ClusterInfo";git push')

In [59]:
# 'gcloud compute instances create instance-1 --project=cellular-hybrid-358402 --zone=us-central1-a --machine-type=e2-medium --network-interface=network-tier=PREMIUM,subnet=default --no-restart-on-failure --maintenance-policy=TERMINATE --provisioning-model=SPOT --instance-termination-action=STOP --service-account=932709639592-compute@developer.gserviceaccount.com --scopes=https://www.googleapis.com/auth/devstorage.read_only,https://www.googleapis.com/auth/logging.write,https://www.googleapis.com/auth/monitoring.write,https://www.googleapis.com/auth/servicecontrol,https://www.googleapis.com/auth/service.management.readonly,https://www.googleapis.com/auth/trace.append --tags=http-server,https-server --create-disk=auto-delete=yes,boot=yes,device-name=instance-1,image=projects/ubuntu-os-cloud/global/images/ubuntu-2204-jammy-v20220810,mode=rw,size=10,type=projects/cellular-hybrid-358402/zones/us-central1-a/diskTypes/pd-balanced --no-shielded-secure-boot --shielded-vtpm --shielded-integrity-monitoring --reservation-affinity=any'

In [60]:
# gcloud compute instances create instance-1 --project=cellular-hybrid-358402 --zone=us-west1-b --machine-type=e2-medium --network-interface=network-tier=PREMIUM,subnet=default --no-restart-on-failure --maintenance-policy=TERMINATE --provisioning-model=SPOT --instance-termination-action=STOP --service-account=932709639592-compute@developer.gserviceaccount.com --scopes=https://www.googleapis.com/auth/devstorage.read_only,https://www.googleapis.com/auth/logging.write,https://www.googleapis.com/auth/monitoring.write,https://www.googleapis.com/auth/servicecontrol,https://www.googleapis.com/auth/service.management.readonly,https://www.googleapis.com/auth/trace.append --tags=http-server,https-server --create-disk=auto-delete=yes,boot=yes,device-name=instance-1,image=projects/ubuntu-os-cloud/global/images/ubuntu-2204-jammy-v20220810,mode=rw,size=10,type=projects/cellular-hybrid-358402/zones/us-central1-a/diskTypes/pd-balanced --no-shielded-secure-boot --shielded-vtpm --shielded-integrity-monitoring --reservation-affinity=any